## **Setting Up SharePoint Integration (Tool) with Azure AI Agents**

### **📝 Overview**

SharePoint can be used as a secure data source to ground your Azure AI Agents with documents stored in SharePoint. For example, you can connect to a SharePoint site like `contoso.sharepoint.com/sites/policies`. When a user sends a query, Azure AI Agents will determine whether SharePoint should be leveraged. If so, the query is sent via the SharePoint tool, which:

1. Verifies if the user has an M365 Copilot license.
2. Uses the end user's identity to retrieve relevant documents they have access to.
3. Generates responses based on the retrieved information.

With SharePoint integration, **On-Behalf-Of (OBO)** authentication is supported, allowing the SharePoint tool to retrieve documents based on the end user's identity and access.

### **✅ Prerequisites**

Before you begin, ensure the following:

1. **Existing SharePoint Site**: Developers must have access to the SharePoint site.
2. **M365 Copilot Licenses**: Both developers and end users must have M365 Copilot licenses.
3. **Supported Regions**: Your Azure OpenAI (AOAI) resource and AI project must be in one of the following regions:
   - `westus`
   - `japaneast`
   - `francecentral`

🔑 Role-Based Access Control (RBAC) Requirements

- **For Developers**: Must have the **AI Developer** role to create, read, update, or delete (CRUD) a SharePoint tool in Azure AI Agent.
- **For End Users**: Must have the **AI Developer** role to enable On-Behalf-Of (OBO) authentication.

### **🛠️ How-To Guide**

1️⃣ Verify Access to the Existing SharePoint Site: Ensure developers have the necessary permissions to access the SharePoint site:

1. Navigate to the SharePoint site.
2. Click on the gear icon (⚙️) in the top-right corner and select **"Site Permissions"**.
3. Choose **"Advanced Permission Settings"**.
4. Click on **"Check Permissions"** in the ribbon.
5. Enter the developer's username or email and click **"Check Now"**.
6. Review the permissions to ensure they have appropriate access.

2️⃣ Confirm M365 Copilot Licenses for Developers and End Users: Both developers and end users must have M365 Copilot licenses. Follow these steps to verify:

**For Administrators:**
1. Sign in to [Microsoft Admin Center](https://admin.microsoft.com/) with an admin account.
2. Navigate to **Billing > Licenses** to view available licenses.  
   👉 [Learn more about licensing requirements](https://learn.microsoft.com/en-us/microsoft-copilot-studio/requirements-licensing).
3. Go to **Users > Active Users**.
4. Select a user and click on **Licenses and Apps**.
5. Ensure the **M365 Copilot** license is assigned.

**Option 1: Check via Microsoft Account Portal**
1. Go to [Microsoft Account Portal](https://myaccount.microsoft.com/).
2. On the left sidebar, check:
   - **My Apps** → Look for M365 Copilot.
   - **My Access** → This may show assigned licenses.
   - **My Groups** → If there is an M365 Copilot group, you might be assigned a license through it.
3. If you see **My Subscriptions** or **Licenses**, click it to check if the **M365 Copilot** license is assigned to you.

> **Tip**: If you can’t find your license, contact your IT department or Microsoft 365 administrator to verify.

3️⃣ Verify Azure OpenAI (AOAI) Resource and AI Project Regions

Ensure your AOAI resources and AI projects are in one of the supported regions:
- **West US**
- **Japan East**
- **France Central**

4️⃣ Assign Appropriate RBAC Roles

Proper RBAC roles are essential for managing SharePoint tools in Azure AI Agent.

**For Developers:**
- Must have the **AI Developer** role to CRUD a SharePoint tool.

**For End Users:**
- Must have the **AI Developer** role to enable OBO authentication.

**Steps to Assign Roles:**
1. Go to [Azure Portal](https://portal.azure.com/).
2. Navigate to **Subscriptions** and select your subscription.
3. Click on **Access Control (IAM)** > **Add Role Assignment**.
4. Choose the **AI Developer** role.
5. Select the users or groups to assign the role to and click **Save**.

Once you've completed these steps..

1. **Create a New Connection**
   - Provide the required key-value pairs for the connection:
   - **`resource-key`**: Your Bing resource key.
   - **`endpoint`**: Your Bing search endpoint (typically provided in your resource details).

2. **Add a Knowledge Source**
   - Click to add a knowledge source and select **Grounding with Bing Search** (UI) or run code below.
   - If you don’t see this option:
      - Verify that your resource has been created in the same resource group as your AI Agent.
      - Ensure you have the necessary permissions.

In [1]:
import os

from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Define the target directory
target_directory = os.getcwd()  # Get the current working directory

# Move one directory back
parent_directory = os.path.dirname(target_directory)

# Check if the parent directory exists
if os.path.exists(parent_directory):
    # Change the current working directory to the parent directory
    os.chdir(parent_directory)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Parent directory {parent_directory} does not exist.")

Directory changed to c:\Users\pablosal\Desktop\gbb-ai-agenticrag


In [2]:
import importlib.metadata as md

# Versions - we are currently 1.0.0b9 of azure-ai-projects
print("semantic-kernel version:", md.version("semantic-kernel"))
# if you want to Upgrade the SDKs, uncomment the line below but code might break
# %pip install -U semantic-kernel azure-ai-projects azure-identity

semantic-kernel version: 1.28.1


In [3]:
import os
import re
import time
import logging
import json
from datetime import datetime as pydatetime
from typing import Any, List, Dict, Optional
from dotenv import load_dotenv
import asyncio
from datetime import timedelta

# Azure AI Projects
from azure.identity.aio import DefaultAzureCredential
from azure.core.exceptions import HttpResponseError

# semantic kernel
from semantic_kernel.contents import AuthorRole
from semantic_kernel.agents.azure_ai.azure_ai_agent import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.open_ai.run_polling_options import RunPollingOptions

# Load environment variables from .env file
load_dotenv()

# configure logging
from utils.ml_logging import get_logger

logger = get_logger()

In [4]:
from azure.core.exceptions import ServiceRequestError
from azure.ai.projects.aio import AIProjectClient

async def get_connection_id(client: AIProjectClient, env_var: str) -> Optional[str]:
    """
    Retrieves the connection object using a connection name stored in an environment variable.

    Args:
        client: The Azure AI Project client.
        env_var (str): The environment variable holding the connection name.

    Returns:
        Connection object if found, otherwise raises an error.
    """
    connection_name = os.getenv(env_var)
    if not connection_name:
        logger.error(f"Missing environment variable: '{env_var}'")
        raise ValueError(f"Environment variable '{env_var}' is required.")

    try:
        connection = await client.connections.get(connection_name=connection_name)
        logger.info(f"Retrieved Connection ID for {env_var}: {connection.id}")
        return connection
    except Exception as e:
        logger.error(f"Failed to retrieve connection for {env_var}: {e}")
        raise

### **Create Client and Load Azure AI Foundry**

Here, we initialize the Azure AI client using DefaultAzureCredential. This allows us to authenticate and connect to the Azure AI service.

In [5]:
project_client = AzureAIAgent.create_client(credential=DefaultAzureCredential(),
conn_str=os.getenv("AZURE_AI_FOUNDRY_CONNECTION_STRING"))

In [ ]:
from azure.ai.projects.models import (
    SharepointTool,
    ToolSet,
)

# Initialize Azure AI Agent settings
dataretrievalagent_settings = AzureAIAgentSettings.create()

toolset = ToolSet()

try:
    # Retrieve and add SharePoint Tool
    sharepoint_connection = await get_connection_id(project_client, "TOOL_CONNECTION_NAME_SHAREPOINT")
    toolset.add(SharepointTool(connection_id=sharepoint_connection.id))

    logger.info("Successfully created ToolSet with SharePoint and Fabric tools.")
except Exception as e:
    logger.error(f"Failed to create ToolSet: {e}")
    raise

The create method is deprecated. Use the __new__ method instead.


In [ ]:
dataretrievalagent_settings_definition = await project_client.agents.create_agent(
    model=dataretrievalagent_settings.model_deployment_name,
    name="SharePointDataRetrievalAgent",
    description=(
        "An AI agent specialized in retrieving and analyzing unstructured documents from SharePoint. "
        "This includes research papers, legal documents, and product engineering files (PDFs). "
        "The agent is designed to assist in research and development by providing accurate, relevant, and actionable insights. "
        "If no relevant data is found, the agent must clearly indicate this and provide suggestions for alternative queries or data sources."
    ),
    instructions=(
        "### Role & Objective\n"
        "You are a research-focused AI assistant responsible for retrieving and analyzing unstructured documents exclusively from SharePoint. "
        "Your goal is to provide precise, well-referenced, and relevant responses to support research and development, legal analysis, and product engineering efforts.\n\n"
        
        "### Data Retrieval & Prioritization\n"
        "1. **Unstructured Data (SharePoint):** \n"
        "   - Retrieve documents from SharePoint when the query involves research papers, legal documents, or product engineering files (PDFs).\n"
        "   - Focus on extracting key insights, summaries, and actionable information from the retrieved documents.\n"
        "   - Example: 'Retrieve research papers on Material X used in high-temperature environments,' or 'Find legal documents related to patent filings for Product A.'\n\n"
        
        "2. **Document Types:** \n"
        "   - Research Papers: Summarize findings, methodologies, and conclusions.\n"
        "   - Legal Documents: Extract key clauses, compliance requirements, and patent-related information.\n"
        "   - Product Engineering Files: Highlight design notes, test results, and engineering decisions.\n\n"
        
        "3. **Integrated Queries:** \n"
        "   - If the query spans multiple document types, retrieve and integrate the results for a comprehensive response.\n"
        "   - Ensure clarity in presenting combined insights.\n\n"
        
        "4. **Fallback Behavior:** \n"
        "   - If no relevant data is found in SharePoint, respond with:\n"
        "     - A clear statement that no relevant data was found.\n"
        "     - Suggestions for alternative queries or data sources (if applicable).\n"
        "     - Example: 'No relevant data was found for the requested query in SharePoint. Consider refining your query or exploring other data sources.'\n\n"
        
        "### Response Quality\n"
        "1. **Accuracy & Relevance:** Always prioritize retrieving the most current and applicable documents from SharePoint.\n"
        "2. **Clarity & Transparency:** Clearly indicate the data source(s) used and any limitations in the available information.\n"
        "3. **Fallback Handling:** If no relevant data is found, provide a professional and helpful fallback response as outlined above.\n"
        "4. **Professionalism:** Present findings in a structured and concise manner to facilitate decision-making.\n"
        "5. **Document Context:** Ensure that extracted insights are presented with sufficient context to maintain their relevance and accuracy.\n"
    ),
    toolset=toolset,
    headers={"x-ms-enable-preview": "true"},
    temperature=0.7,
    top_p=1,
    metadata={
        "use_case": "Unstructured Data Retrieval for R&D, Legal, and Engineering",
        "data_source": "SharePoint",
    },
)

# Print the agent's run ID (agent ID)
print(f"SharePointDataRetrievalAgent Run ID: {dataretrievalagent_settings_definition.id}")

In [ ]:
user_query = "In which glucose ranges does Product A underperform compared to Product B, and what clinical impact could this have?"
sharepoint_response, threadID = await run_agent(project_client, "asst_x73A5QElh86JNelOI50PgH7T", user_query)